In [1]:
import json
import os
import bson
import pandas as pd
import numpy as np

from clickhouse_driver import Client

In [2]:
data = pd.read_csv("../data/en.openfoodfacts.org.products.csv", sep="\t", nrows=50000)
data = data.drop(columns = list(data.select_dtypes(['object']).columns))
data = data.dropna(axis=1, how='all')

/tmp/ipykernel_555157/4293239321.py:1: DtypeWarning: Columns (0,8,14,20,21,22,28,29,30,32,69) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../data/en.openfoodfacts.org.products.csv", sep="\t", nrows=50000)


In [3]:
list(data.columns)

['created_t',
 'last_modified_t',
 'serving_quantity',
 'additives_n',
 'nutriscore_score',
 'nova_group',
 'ecoscore_score',
 'product_quantity',
 'unique_scans_n',
 'completeness',
 'last_image_t',
 'energy-kj_100g',
 'energy-kcal_100g',
 'energy_100g',
 'energy-from-fat_100g',
 'fat_100g',
 'saturated-fat_100g',
 'monounsaturated-fat_100g',
 'polyunsaturated-fat_100g',
 'omega-3-fat_100g',
 'omega-6-fat_100g',
 'omega-9-fat_100g',
 'trans-fat_100g',
 'cholesterol_100g',
 'carbohydrates_100g',
 'sugars_100g',
 'starch_100g',
 'polyols_100g',
 'fiber_100g',
 'soluble-fiber_100g',
 'insoluble-fiber_100g',
 'proteins_100g',
 'nucleotides_100g',
 'salt_100g',
 'sodium_100g',
 'alcohol_100g',
 'vitamin-a_100g',
 'vitamin-d_100g',
 'vitamin-e_100g',
 'vitamin-k_100g',
 'vitamin-c_100g',
 'vitamin-b1_100g',
 'vitamin-b2_100g',
 'vitamin-pp_100g',
 'vitamin-b6_100g',
 'vitamin-b9_100g',
 'folates_100g',
 'vitamin-b12_100g',
 'biotin_100g',
 'pantothenic-acid_100g',
 'bicarbonate_100g',
 'pot

In [4]:
np.unique([v for k, v in dict(data.dtypes).items()])

array([dtype('int64'), dtype('float64')], dtype=object)

In [5]:
# dict(zip(columns_name_old, columns_name_new))

In [6]:
# data = data.fillna(0)
columns_name_old = list(data.columns)
columns_name_new = list(map(lambda x: x.replace("-", "_"), columns_name_old))

data = data.rename(columns=dict(zip(columns_name_old, columns_name_new)))

In [14]:
client = Client(host='localhost', port='9000')
client.execute('SHOW DATABASES')

[('INFORMATION_SCHEMA',), ('default',), ('information_schema',), ('system',)]

In [15]:
table_name = "Market"
query = f"CREATE TABLE {table_name}"


columns = []
for k, v in dict(data.dtypes).items():
    if v == "int64":
        v = "Int64"
    elif v == "object":
        v = "String"
    elif v == "float64":
        v = "Float"
    columns.append(f"{k} {v}")
columns_name = ", ".join(columns)
columns_name = "(" + columns_name + ")"
query = query + columns_name + " Engine = Memory"

In [25]:
from clickhouse_driver import Client


client = Client(host='localhost')
# client.execute(query)
# client.execute(f"INSERT INTO {table_name} VALUES", data.to_dict('records'))

In [26]:
data = client.execute(f"SELECT * FROM {table_name}")

In [15]:
# data = pd.read_parquet("../experiments/market_data.parquet")

In [8]:
from sklearn.cluster import KMeans

model = KMeans(n_clusters=10)

In [9]:
model.fit(data)

KMeans(n_clusters=10)

In [24]:
import pandas as pd
from sklearn.datasets import load_iris
from pyspark.sql import SparkSession

from pyspark import SparkContext, SparkConf
from pyspark.ml.clustering import KMeans

df_iris = load_iris(as_frame=True)

In [25]:
config = SparkConf()
sc = SparkContext(conf=config)
spark = SparkSession(sc)

23/01/08 15:59:14 WARN Utils: Your hostname, dkrivenkov resolves to a loopback address: 127.0.1.1; using 192.168.1.218 instead (on interface wlp146s0)
23/01/08 15:59:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/08 15:59:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [26]:
pd_df_iris = pd.DataFrame(df_iris.data, columns = df_iris.feature_names)
pd_df_iris['target'] = pd.Series(df_iris.target)

In [27]:
spark_df_iris = spark.createDataFrame(pd_df_iris)
spark_df_iris = spark_df_iris.drop("target")

/home/dkrivenkov/anaconda3/envs/env39/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/dkrivenkov/anaconda3/envs/env39/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [28]:
from pyspark.ml.feature import VectorAssembler
assemble=VectorAssembler(inputCols=[
'sepal length (cm)',
'sepal width (cm)',
'petal length (cm)',
'petal width (cm)'],outputCol = 'iris_features')

assembled_data=assemble.transform(spark_df_iris)

In [30]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

KMeans_=KMeans(featuresCol='iris_features', k=10)

KMeans_fit=KMeans_.fit(assembled_data)

KMeans_transform=KMeans_fit.transform(assembled_data) 

In [68]:
d = KMeans_transform.toPandas()
d["a"] = KMeans_transform.select("prediction").toPandas().values.flatten()

In [70]:
d.dtypes

sepal length (cm)    float64
sepal width (cm)     float64
petal length (cm)    float64
petal width (cm)     float64
iris_features         object
prediction             int32
a                      int32
dtype: object

In [65]:
"+".join(map(str, KMeans_transform.select("prediction").toPandas().values.flatten()))
# KMeans_transform.select("prediction").toPandas().values.flatten()

'5+1+1+1+5+5+1+5+1+1+5+1+1+1+5+5+5+5+5+5+5+5+1+5+1+1+5+5+5+1+1+5+5+5+1+1+5+5+1+5+5+1+1+5+5+1+5+1+5+1+4+4+4+0+4+0+4+6+4+0+6+0+0+4+0+4+0+0+4+0+4+0+4+4+4+4+4+4+4+0+0+0+0+4+0+4+4+4+0+0+0+4+0+6+0+0+0+4+6+0+7+4+8+9+7+2+0+8+9+8+7+9+7+4+9+7+9+3+2+4+7+4+2+4+7+8+4+4+9+8+8+3+9+4+9+2+7+9+4+7+7+7+4+7+7+7+4+9+7+4'

In [52]:
KMeans_transform.select("prediction").collect.map(_.getDouble(0))

AttributeError: 'function' object has no attribute 'map'